# DCL-Demo

Typische Rechte für die Arbeit mit Daten:

| Aktion | Benötigte Berechtigung |
| --- | --- |
| Tabelle zeigen | SELECT |
| Werte ändern | UPDATE |
| Werte löschen | DELETE |
| View anzeigen | SELECT |
| Procedure ausführen | EXECUTE |
| Function ausführen | EXECUTE |

## Testuser J\_Doe

Wir erstellen zum Ausprobieren der Berechtigungen den Testuser J\_Doe ohne Login.

J\_Doe bekommt initial SELECT Recht in Schema\_B.

Dadurch kann er mittels der Views in Schema\_B Daten aus den Tabellen in Schema\_A anzeigen.

Die SELECT-Berechtigung auf das Schema\_B erlaubt die Nutzung aller im Schema\_B enthaltenen Views.

Mit DENY kann dennoch der Zugriff auf einzelne Views im Schema\_B verweigert werden.

Mit REVOKE können die erteilten Rechte

In [ ]:
USE DCL_Demo;
CREATE USER J_Doe WITHOUT LOGIN;
GRANT SELECT ON SCHEMA::Schema_B TO J_Doe;

In [ ]:
-- Keine Berechtigung zum Zugriff auf Tabellen in Schema_A
EXECUTE AS USER = 'J_Doe';
SELECT p.ID, p.Vorname, p.Nachname, k.Kategorie
FROM Schema_A.Kategorie AS k
JOIN Schema_A.Person AS p ON k.ID = p.KategorieID
REVERT;

In [ ]:
-- Darf ein VIEW in Schema_B anzeigen
EXECUTE AS USER = 'J_Doe';
SELECT * FROM Schema_B.vw_Sport;
REVERT;

In [ ]:
-- Darf Prozedur nicht ausführen
EXECUTE AS USER = 'J_Doe';
EXEC Schema_B.usp_FindePerson 'Micha';
REVERT;

In [ ]:
-- Darf Funktion nicht ausführen
EXECUTE AS USER = 'J_Doe';
SELECT Schema_B.udf_CountPeople('Sport') AS [Anzahl Personen in Sport];
REVERT;

In [ ]:
-- Verweigere Anzeige des Views Kino
DENY SELECT ON Schema_B.vw_Kino TO J_Doe;

In [ ]:
EXECUTE AS USER = 'J_Doe';
SELECT * FROM Schema_B.vw_IT; -- ok
SELECT * FROM Schema_B.vw_Kino; -- nicht erlaubt
REVERT;

In [ ]:
-- SELECT Recht in Schema_B wird entzogen
REVOKE SELECT ON SCHEMA::Schema_B FROM J_Doe;

In [ ]:
EXECUTE AS USER = 'J_Doe';
SELECT * FROM Schema_B.vw_IT; -- verweigert
REVERT;

In [ ]:
-- EXECUTE Recht wird erteilt
GRANT EXECUTE ON SCHEMA::Schema_B TO J_Doe;

In [ ]:
EXECUTE AS USER = 'J_Doe';
SELECT * FROM Schema_B.vw_Sport; -- verweigert
REVERT;

In [ ]:
-- Ausführen der Funktion ist nun erlaubt
EXECUTE AS USER = 'J_Doe';
SELECT Schema_B.udf_CountPeople('Sport') AS [Anzahl Personen in Sport];
REVERT;

In [ ]:
-- Ausführen der Prozedur ist nun erlaubt
EXECUTE AS USER = 'J_Doe';
EXEC Schema_B.usp_FindePerson 'Al';
REVERT;

In [ ]:
-- Entziehe auch das EXECUTE Recht wieder
REVOKE EXECUTE ON Schema_B.vw_Sport FROM J_Doe;